HOW THE HELL DO I MAKE A GNN

In [1]:
import torch
import torch.nn as nn
from torch_geometric.nn import MessagePassing

Graph neural network

$v_i' = \phi^v (v_i, \sum_j \phi^e(v_i, v_j))$

$\phi^e$ is the edge model which creates the edge message from two nodes

$\phi^v$ is the node model which predicts the instantaneous acceleration of the recieving node from the aggregated edge messages and node information.

Both are MLPs.

The node information $x_i$ is the position $(x, y)$, velocity $(\Delta x, \Delta y)$, charge and mass.



In [2]:
def get_edge_index(num_nodes): #edge index for fully connected graph
    idx = torch.arange(num_nodes)
    edge_index = torch.cartesian_prod(idx, idx)
    edge_index = edge_index[edge_index[:, 0] != edge_index[:, 1]]
    return edge_index.t()

edges =get_edge_index(10)
edges.shape

torch.Size([2, 90])

In [ ]:
def get_edge_index(num_nodes): #edge index for fully connected graph
    idx = torch.arange(num_nodes)
    edge_index = torch.cartesian_prod(idx, idx)
    edge_index = edge_index[edge_index[:, 0] != edge_index[:, 1]]
    return edge_index.t()

class NBodyGNN(MessagePassing):
    def __init__(self, node_dim = 6, acc_dim = 2, hidden_dim = 300):
        """ 
        N-body graph NN class.

        Args:
            node_dim (int): dimensionality of the node (in 2d case it is 6)
            acc_dim (int): dimensionality of the output of the network, which are accelerations so in 2d = 2
            hidden_dim (int): hidden layer dimensions 

        """
    
        super().__init__(aggr='add')
         
        #edge model MLP
        self.edge_model = nn.Sequential(
            nn.Linear(2*node_dim, hidden_dim), #inputs = node information for two nodes
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
            nn.Linear(hidden_dim, 100) #output message features of dimension 100 for standard and L1 model 
        )
        #node model MLP
        self.node_model = nn.Sequential(
            nn.Linear(node_dim + 100, hidden_dim), #inputs = sum of outputs of edge model and node features
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
            nn.ReLU()
            nn.Linear(hidden_dim, acc_dim) #output = predicted acc
        )

    def message(self, x_i, x_j):
        x = torch.cat((x_i, x_j), dim = -1) #can change the dimension ? not hugely sure which one it right tbh 
        x = self.edge_model(x)
        return x


    def forward(self, x, edge_index): 
        """forward pass of this network

        Args:
            x (torch.Tensor): shape is [num nodes, node features]
            edge_index (torch.Tensor): shape is [2, num edges]
        """

        edge_message =  self.propagate(edge_index, x = (x,x)) #use same feature matrix for both source and target nodes
        acc = self.node_model(edge_message) #predict accelerations

        return acc


        

        


#TODO: make the forward pass of the model where u predict things
# get loss function with losses (only L2 for now later we do L1 for the message features)




